In [32]:
# -*- coding: utf-8 -*
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import os
import multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
from tqdm import tqdm
from tensors_dataset_path import TensorDatasetPath
from tensors_dataset_img import TensorDatasetImg
import random
import sys
from utils import *
from models import *
from data_transform import *

In [62]:
# Setup reprouducible environment

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(False)

setup_seed(20)

In [54]:
!python knowledge_distill_dataset.py
!python data_compression.py

model_name:  resnets
checkpoint:  resnets_clean
Files already downloaded and verified
[2023-06-22 15:11:04.510 pytorch-1-12-gpu-py-ml-g4dn-xlarge-60d67c819515f85736f0b2ea671f:339 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.24b20230214-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.24b20230214-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
[2023-06-22 15:11:04.981 pytorch-1-12-gpu-py-ml-g4dn-xlarge-60d67c819515f85736f0b2ea671f:339 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
^C
Traceback (most recent call last):
  File "data_compression.py", line 27, in <module>
    dataset = torch.load("./dataset/distill_" + distill_data_name)
  File "/opt/conda/lib/python3.8/site

In [64]:
params = read_config()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [63]:
model_name = params["model"]
model_set = {
    "resnets": ResNetS(nclasses=10),
    "vgg_face": VGG_16(),
    "gtsrb": gtsrb(),
    "resnet50": models.resnet50(),
}
print("model_name: ", model_name)
model = model_set[model_name]

ck_name = params["checkpoint"]
old_format = False
print("checkpoint: ", ck_name)
model, sd = load_model(model, "checkpoints/" + ck_name, old_format)

model_name:  resnets
checkpoint:  resnets_clean


In [65]:
if torch.cuda.is_available():
    model = model.cuda()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
model.to(device)

for name, value in model.named_parameters():
    if name == "layer4.0.conv1.weight":
        break
    value.requires_grad = False

model.eval()

print('model loaded')

model loaded


In [67]:
# Load training dataset

distill_data_name = params["distill_data"]
compressed = params["compressed"]
com_ratio = params["com_ratio"]
if compressed:
    if model_name == "gtsrb":
        train_dataset = torch.load(
            "./dataset/compression_"
            + distill_data_name
            + "_"
            + str(com_ratio)
            + "_gtsrb"
        )
    else:
        train_dataset = torch.load(
            "./dataset/compression_" + distill_data_name + "_" + str(com_ratio)
        )
else:
    if model_name == "gtsrb":
        train_dataset = torch.load("./dataset/distill_" + distill_data_name + "_gtsrb")
    else:
        train_dataset = torch.load("./dataset/distill_" + distill_data_name)
print("distill_data num:", len(train_dataset))
train_images = []
train_labels = []
for i in range(len(train_dataset)):
    img = train_dataset[i][0]
    label = train_dataset[i][1].cpu()
    train_images.append(img)
    train_labels.append(label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# train_images = np.load('train_images.npy', allow_pickle = True)
# train_labels = np.load('train_images.npy', allow_pickle = True)
print("load train data finished")

print(type(train_images), type(train_images[0]))
print(type(train_labels), type(train_labels[0]))

distill_data num: 20000
load train data finished
<class 'numpy.ndarray'> <class 'PIL.Image.Image'>
<class 'numpy.ndarray'> <class 'torch.Tensor'>


In [68]:
# Load test dataset

dataset_name = params["data"]

if dataset_name == "VGGFace":
    test_images, test_labels = get_dataset_vggface("./dataset/VGGFace/", max_num=10)
elif dataset_name == "tiny-imagenet-200":
    testset = torchvision.datasets.ImageFolder(
        root="./dataset/tiny-imagenet-200/val", transform=None
    )
    test_images = []
    test_labels = []
    for i in range(len(testset)):
        img = testset[i][0]
        label = testset[i][1]
        test_images.append(img)
        test_labels.append(label)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
elif dataset_name == "cifar10":
    _dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True)
    test_images = [_dataset[i][0] for i in range(len(_dataset))]
    test_labels = _dataset.targets
else:
    test_images, test_labels = get_dataset("./dataset/" + dataset_name + "/test/")


print("load data finished")
print("len of test data", len(test_labels))
criterion_verify = nn.CrossEntropyLoss()

Files already downloaded and verified
load data finished
len of test data 10000


In [69]:
batch_size = 320

if model_name == "resnets":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=cifar100_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=cifar10_transforms_test,
            mode="test",
            test_poisoned="False",
            transform_name="cifar10_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=0,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=cifar10_transforms_test,
            mode="test",
            test_poisoned="True",
            transform_name="cifar10_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=0,
        pin_memory=True,
    )
elif model_name == "vgg_face":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=LFW_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetPath(test_images, test_labels, mode="test", test_poisoned="False"),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetPath(test_images, test_labels, mode="test", test_poisoned="True"),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )
elif model_name == "gtsrb":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=cifar100_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetPath(
            test_images,
            test_labels,
            transform=gtsrb_transforms,
            mode="test",
            test_poisoned="False",
            transform_name="gtsrb_transforms",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetPath(
            test_images,
            test_labels,
            transform=gtsrb_transforms,
            mode="test",
            test_poisoned="True",
            transform_name="gtsrb_transforms",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

elif model_name == "resnet50":
    train_loader = DataLoader(
        TensorDatasetImg(train_images, train_labels, transform=imagenet_transforms),
        shuffle=True,
        batch_size=batch_size,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=imagenet_transforms,
            mode="test",
            test_poisoned="False",
            transform_name="imagenet_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

    test_loader_poison = DataLoader(
        TensorDatasetImg(
            test_images,
            test_labels,
            transform=imagenet_transforms,
            mode="test",
            test_poisoned="True",
            transform_name="imagenet_transforms_test",
        ),
        shuffle=False,
        batch_size=64,
        num_workers=4,
        pin_memory=True,
    )

print("poison data finished")

poison data finished


In [70]:
lr = params["lr"]
epochs = params["epochs"]

# optimizer_poison = optim.SGD(model.parameters(), lr=lr)
# scheduler_poison = lr_scheduler.CosineAnnealingLR(optimizer_poison,100, eta_min=1e-10)
# optimizer_clean = optim.SGD(model.parameters(), lr=lr/2*1.0)
# scheduler_clean = lr_scheduler.CosineAnnealingLR(optimizer_clean,100, eta_min=1e-10)
optimizer = optim.SGD(model.parameters(), lr=lr)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-10)
criterion = nn.MSELoss()

###########------------First Accuracy----------------############
print("first accuracy:")
before_clean_acc = validate(model, -1, test_loader, criterion_verify, True)
before_poison_acc = validate(model, -1, test_loader_poison, criterion_verify, False)

first accuracy:
epoch: -1
clean accuracy: 0.9039
epoch: -1
attack accuracy: 0.1031


In [ ]:
lambda1 = 1
alpha = 0.05

import warnings
warnings.filterwarnings("ignore") 

# TODO: remove this
epochs = 1000

for epoch in tqdm(range(epochs)):
    # train_with_grad_control(model, epoch, train_loader_clean, criterion, optimizer)
    # train_with_grad_control(model, epoch, train_loader, criterion, optimizer)

    print("lambda1: ", lambda1)
    adjust = train_with_grad_control(
        model, epoch, train_loader, criterion, optimizer, lambda1
    )
    lambda1 += alpha * adjust
    lambda1 = min(lambda1, 1)
    lambda1 = max(0, lambda1)

    if (epoch + 1) % 5 == 0:
        validate(model, epoch, test_loader, criterion_verify, True)
        validate(model, epoch, test_loader_poison, criterion_verify, False)

    state = {
        "net": model.state_dict(),
        "masks": [w for name, w in model.named_parameters() if "mask" in name],
        "epoch": epoch,
        # 'error_history': error_history,
    }
    torch.save(state, "checkpoints/cifar10_optim_1.t7")
    scheduler.step()

print("model train finished")

  0%|          | 0/1000 [00:00<?, ?it/s]

lambda1:  1


  0%|          | 1/1000 [00:07<1:57:38,  7.07s/it]

epoch: 0 train loss: 33.91948876953125
lambda1:  1


  0%|          | 2/1000 [00:14<1:58:40,  7.14s/it]

...

epoch: 807 train loss: 1.0692960796356201
lambda1:  0.4197207654210656


 81%|████████  | 809/1000 [2:01:05<24:56,  7.83s/it]

epoch: 808 train loss: 1.0505843086242677
lambda1:  0.4197666928978513


 81%|████████  | 809/1000 [2:01:10<28:36,  8.99s/it]


KeyboardInterrupt: 